In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import seaborn as sns

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Generate data as in the example
n_samples = 20000
y = np.random.uniform(-.5, 2.5, n_samples)
X = y.reshape(-1, 1)  # Reshape y into a 2D array (n_samples, 1)
xtrue = np.zeros_like(y)

# Apply conditions to generate xtrue based on y
for i in range(len(xtrue)):
    if (0 < y[i] < 0.5) or (1.5 < y[i] < 2):
        xtrue[i] = np.random.normal(1, scale=0.5)
    else:
        xtrue[i] = np.random.normal(-1, scale=0.5)

# Generate varying observation noise
obs_noise_std = np.random.uniform(0.5, 1.5, len(xtrue))
x = xtrue + np.random.normal(0, obs_noise_std, len(xtrue))

# Convert to PyTorch tensors
X = torch.tensor(X, dtype=torch.float32)
x = torch.tensor(x, dtype=torch.float32).reshape(-1, 1)

# Conditional Normalizing Flow Layer
class ConditionalCouplingLayer(nn.Module):
    def __init__(self, condition_dim, data_dim, hidden_dims=[64, 64]):
        super().__init__()
        
        # Total input dimension (condition + data)
        total_dim = condition_dim + data_dim
        
        # Scale network
        scale_layers = []
        prev_dim = total_dim
        for h_dim in hidden_dims:
            scale_layers.append(nn.Linear(prev_dim, h_dim))
            scale_layers.append(nn.ReLU())
            prev_dim = h_dim
        scale_layers.append(nn.Linear(prev_dim, data_dim))
        self.scale_net = nn.Sequential(*scale_layers)
        
        # Shift network
        shift_layers = []
        prev_dim = total_dim
        for h_dim in hidden_dims:
            shift_layers.append(nn.Linear(prev_dim, h_dim))
            shift_layers.append(nn.ReLU())
            prev_dim = h_dim
        shift_layers.append(nn.Linear(prev_dim, data_dim))
        self.shift_net = nn.Sequential(*shift_layers)
    
    def forward(self, condition, x, inverse=False):
        # Combine condition and data
        z = torch.cat([condition, x], dim=1)
        
        # Compute scale and shift
        scale = torch.sigmoid(self.scale_net(z) + 1)  # Ensure positive scale
        shift = self.shift_net(z)
        
        if not inverse:
            # Forward transformation
            x_transformed = x * scale + shift
            log_det = torch.log(scale).sum(dim=1)
            return x_transformed, log_det
        else:
            # Inverse transformation
            x_inv = (x - shift) / scale
            log_det = -torch.log(scale).sum(dim=1)
            return x_inv, log_det

# Conditional Normalizing Flow Model
class ConditionalDensityEstimator(nn.Module):
    def __init__(self, condition_dim, data_dim, n_flows=3, hidden_dims=[64, 64]):
        super().__init__()
        
        # Create multiple flow layers
        self.flows = nn.ModuleList([
            ConditionalCouplingLayer(condition_dim, data_dim, hidden_dims) 
            for _ in range(n_flows)
        ])
        
        # Base distribution (standard normal)
        self.base_dist = torch.distributions.Normal(
            torch.zeros(data_dim), 
            torch.ones(data_dim)
        )
    
    def forward(self, condition, x):
        # Initial log probability from base distribution
        z = x
        log_prob = self.base_dist.log_prob(z).sum(dim=1)
        
        # Apply transformations
        for flow in self.flows:
            z, log_det = flow(condition, z)
            log_prob += log_det
        
        return log_prob
    
    def sample(self, condition, num_samples=1):
        # Sample from base distribution
        z = self.base_dist.sample((num_samples,))
        
        # Apply inverse transformations
        for flow in reversed(self.flows):
            z, _ = flow(condition, z, inverse=True)
        
        return z

# Prepare data
X_train = X
x_train = x

# Instantiate the model
model = ConditionalDensityEstimator(
    condition_dim=1,  # y dimension
    data_dim=1        # x dimension
)

# Loss function (negative log-likelihood)
def loss_fn(model, X, x):
    log_prob = model(X, x)
    return -log_prob.mean()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
n_epochs = 200
batch_size = 256

for epoch in range(n_epochs):
    # Shuffle data
    indices = torch.randperm(len(X_train))
    X_shuffled = X_train[indices]
    x_shuffled = x_train[indices]
    
    # Mini-batch training
    for i in range(0, len(X_train), batch_size):
        batch_X = X_shuffled[i:i+batch_size]
        batch_x = x_shuffled[i:i+batch_size]
        
        # Zero gradients
        optimizer.zero_grad()
        
        # Compute loss
        loss = loss_fn(model, batch_X, batch_x)
        
        # Backpropagation
        loss.backward()
        
        # Optimize
        optimizer.step()
    
    # Print loss periodically
    if (epoch + 1) % 20 == 0:
        print(f'Epoch [{epoch+1}/{n_epochs}], Loss: {loss.item():.4f}')

# Visualization of conditional distributions
plt.figure(figsize=(15, 5))

# Different conditions to sample from
conditions = [0.25, 1.0, 1.75]

for i, cond in enumerate(conditions, 1):
    # Generate samples for this condition
    cond_tensor = torch.tensor([[cond]], dtype=torch.float32)
    
    # Generate multiple samples
    samples = model.sample(cond_tensor, num_samples=1000).detach().numpy()
    
    plt.subplot(1, 3, i)
    sns.histplot(samples.flatten(), kde=True)
    plt.title(f'Conditional Distribution at y = {cond}')
    plt.xlabel('x')
    plt.ylabel('Density')

plt.tight_layout()
plt.show()

# Print some diagnostics
print("\nModel Training Completed.")
print("Conditions tested:", conditions)

Epoch [20/200], Loss: 2.3336
Epoch [40/200], Loss: 2.7396
Epoch [60/200], Loss: 1.6735
Epoch [80/200], Loss: 2.3300
Epoch [100/200], Loss: 1.8817
Epoch [120/200], Loss: 1.8595
Epoch [140/200], Loss: 2.1415
Epoch [160/200], Loss: 1.7789
Epoch [180/200], Loss: 1.9969
Epoch [200/200], Loss: 2.3197


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 1 but got size 1000 for tensor number 1 in the list.

<Figure size 1500x500 with 0 Axes>

In [5]:
def sample(self, condition, num_samples=1):
    # Repeat the condition to match the batch size
    condition_repeated = condition.repeat(num_samples, 1)
    
    # Sample from base distribution
    z = self.base_dist.sample((num_samples,))
    
    # Apply inverse transformations
    for flow in reversed(self.flows):
        z, _ = flow(condition_repeated, z, inverse=True)
    
    return z
# Visualization of conditional distributions
plt.figure(figsize=(15, 5))

# Different conditions to sample from
conditions = [0.25, 1.0, 1.75]

for i, cond in enumerate(conditions, 1):
    # Generate samples for this condition
    cond_tensor = torch.tensor([[cond]], dtype=torch.float32)
    
    # Generate multiple samples
    samples = model.sample(cond_tensor, num_samples=1000).detach().numpy()
    
    plt.subplot(1, 3, i)
    sns.histplot(samples.flatten(), kde=True)
    plt.title(f'Conditional Distribution at y = {cond}')
    plt.xlabel('x')
    plt.ylabel('Density')

plt.tight_layout()
plt.show()

# Print some diagnostics
print("\nModel Training Completed.")
print("Conditions tested:", conditions)

RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 1 but got size 1000 for tensor number 1 in the list.

<Figure size 1500x500 with 0 Axes>